# Read Libs

In [92]:
import pandas as pd
import datetime as dt
import numpy as np
import os
import regex as re

# display and output settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

display(HTML('''
<style>
    .container { width:85% !important; }
</style>
'''))

pd.set_option('display.min_rows', 20)

# Transform Data

## Clean Scores Data

### Reading In Data and Selecting Columns

In [93]:
scores_df_1 = pd.read_csv('../data/raw_data/tournament_scores/batch_1/2024-09-22 15h28m07s/tournament_scores.csv')
scores_df_2 = pd.read_csv('../data/raw_data/tournament_scores/batch_2/2024-09-22 14h03m51s/tournament_scores.csv')
scores_df = pd.concat([scores_df_1, scores_df_2])


/Users/Mishaun_Bhakta/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [94]:
cols_scores = [
    "event_id",
    "POS",
    "PLAYER",
    "SCORE",
    "R1",
    "R2",
    "R3",
    "R4",
    "TOT",
    "EARNINGS",
    "FEDEX PTS"
]

In [95]:
scores_df = scores_df[cols_scores]

### identifying events that did not have complete score information

In [96]:
# creatiing a dataframe with the number of players per event
scores_df_agg = scores_df.groupby('event_id').count()


* will remove events that had less than 5 players in the scores table

In [97]:
# getting event ids that had more than 5 player threshold 
players_threshold_minimum = 5
keep_eventids = list(scores_df_agg[scores_df_agg['POS']>players_threshold_minimum]\
                     .reset_index()['event_id'])

In [98]:
scores_df = scores_df[scores_df['event_id'].isin(keep_eventids)]

In [99]:
def get_par(total_strokes, to_par_score):
    try:
        par_total = int(total_strokes) - int(to_par_score)
        par = par_total / 4

        # check if par is between 69 and 74 and is a whole number
        if par <= 74 and par >= 69 and par % 1 == 0:
            return par
        else:
            return np.nan
    except:
        return np.nan

In [100]:
winners = scores_df[scores_df['POS'] == "1"].copy()
winners['par'] = winners.apply(lambda row: get_par(row['TOT'], row['SCORE']), axis=1)

In [101]:
par = winners[['event_id', 'par']].drop_duplicates().sort_values('event_id')


In [102]:
scores_df = pd.merge(scores_df, par, on='event_id', how='left')

In [103]:
scores_df

,event_id,POS,PLAYER,SCORE,R1,R2,R3,R4,TOT,EARNINGS,FEDEX PTS,par
0,2,1,Jim Furyk,-18,69,69,69,67,274,"$630,000",NaN,73.0
1,2,2,Rory Sabbatini,-17,69,69,65,72,275,"$380,000",NaN,73.0
2,2,T3,Ernie Els,-16,68,66,73,69,276,"$203,000",NaN,73.0
3,2,T3,Vijay Singh,-16,71,67,67,71,276,"$203,000",NaN,73.0
4,2,5,John Huston,-15,74,67,69,67,277,"$140,000",NaN,73.0
5,2,6,Rocco Mediate,-14,70,69,70,69,278,"$126,000",NaN,73.0
6,2,7,David Duval,-13,73,71,65,70,279,"$118,000",NaN,73.0
7,2,T8,Tiger Woods,-12,70,73,68,69,280,"$99,000",NaN,73.0
8,2,T8,Michael Clark II,-12,69,70,72,69,280,"$99,000",NaN,73.0
9,2,T8,Justin Leonard,-12,67,73,69,71,280,"$99,000",NaN,73.0


## Clean Tournament Details

In [104]:
details_df_1 = pd.read_csv('../data/raw_data/tournament_info/batch_1/2024-09-15 15h13m10s/tournament_info.csv')
details_df_2 = pd.read_csv('../data/raw_data/tournament_info/batch_2/2024-09-22 16h43m40s/tournament_info.csv')

#filtering batch 1 details 
details_df_1 = details_df_1[details_df_1['event_id'] <= 3756]

#filtering batch 2 details to be event id's greater than batch 1's last id
details_df_2 = details_df_2[details_df_2['event_id'] > 3756]

details_df = pd.concat([details_df_1, details_df_2])

/Users/Mishaun_Bhakta/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [105]:
# filtering out null or no detail titles
details_df = details_df[~details_df['tournament_title'].isnull()]
details_df = details_df[details_df['tournament_title'] != "No details found"]

In [106]:
# Replacing all instances of "No details found" with None
details_df.replace("No details found", "", inplace=True)

### Adding Fields to Details

In [107]:
# adding city to details 
details_df['city'] = details_df['golf_course'].apply(lambda x: x.split("-")[-1])

### Cleanup fields to get purse, par, yards, city, and golf course

In [108]:
# define function to clean the purse column
def clean_purse(row_value):
    """
    Cleans the purse value from a given row.

    This function takes a row value, removes any commas, and extracts the first 
    dollar amount found. If no dollar amount is found, it returns NaN.

    Parameters:
    row_value (str): The value from the row which may contain a dollar amount.

    Returns:
    str or float: The cleaned dollar amount as a string without the dollar sign, 
                  or NaN if no dollar amount is found.
    """

    row_value_clean = str(row_value).replace(",", "")

    matches = re.findall("\$\d+", row_value_clean)

    if matches:
        return matches[0].replace("$", "") 
    else:
        return np.nan

# apply the clean_purse function to the purse column
details_df['purse'] = details_df['purse'].apply(clean_purse)


In [109]:
# define function to get yards
def clean_course_yards(row_value):

    matches = re.findall("Yards.*", row_value)

    if matches:
        return int(matches[0].replace("Yards", "") )
    else:
        return np.nan
    

def clean_course_par(row_value):

    matches = re.findall("Par\d{2}", row_value)

    if matches:
        return int(matches[0].replace("Par", "") )
    else:
        return np.nan


In [110]:
# applying functions to create par and yards fields
details_df['Yards'] = details_df['course_details'].apply(clean_course_yards)
details_df['Par'] = details_df['course_details'].apply(clean_course_par)


#### Bring in Par from Scores Data to Get optimal fill rate

In [111]:
par

,event_id,par
0,2,73.0
33,3,72.0
189,4,70.0
334,5,71.0
466,6,72.0
646,7,NaN
802,8,NaN
931,9,72.0
1075,10,72.0
1220,11,72.0


In [113]:
# get par from scores data source
details_df = pd.merge(details_df, 
                      # renaming par column to avoid conflict
                      par.rename(columns={'par': 'par_from_scores'}),
                        on='event_id',
                          how='left'
                    )

In [117]:
# fill in par from scores if par is null
details_df['par_final'] = details_df['Par'].combine_first(details_df['par_from_scores'])

In [118]:

# test = "Evian Resort Golf Club - Évian-les-Bains, France".split("-")
# test.pop(-1)
# test
# test = list(map(lambda x: x.strip(), test))
# test
# " ".join(test)

def clean_golf_course_name(course_name):
    """
    Cleans the golf course name by removing the location part.

    Args:
        course_name (str): The full name of the golf course including location.

    Returns:
        str: The cleaned golf course name without the location.
    """
    # split course name by hyphen
    parts = course_name.split("-")
    
    # remove the last element because this will be the city name
    parts.pop(-1)

    # strip whitespace from each part and join them back together
    parts = list(map(lambda x: x.strip(), parts))
    return " ".join(parts)


details_df['golf_course_clean'] = details_df['golf_course'].apply(clean_golf_course_name)

### Extract start and end dates of tournament

In [119]:
# # logic to build out selecitng start and end dates of events
# # has to handle 2 scenarios , same month tournaments or tournaments overalaping months

# ### need to create dates start and finish for event date
# test_date = "January 4 - 7, 2001"
# test_date2 = "Mar 29 - April 1, 2001"
# event_date = test_date2

# pattern = r"\w+ \d+ - \w+ \d+, \d{4}"

# if len(re.findall(pattern, event_date)) == 0:
#     parts = test_date.split(",")
#     year = parts[-1].strip()
#     month = parts[0].split(" ")[0]

#     start_day = re.findall("\d", parts[0])[0]
#     end_day = re.findall("\d", parts[0])[1]

#     start_date = f"{month} {start_day}, {year}"
#     end_date = f"{month} {end_day}, {year}"

#     print(start_date, end_date)

# elif len(re.findall(pattern, event_date)) > 0:
#     parts = event_date.split(",")
#     year = parts[-1].strip()

#     start_month = parts[0].split(" - ")[0].split(' ')[0]
#     start_day = parts[0].split(" - ")[0].split(' ')[1]

#     end_month = parts[0].split(" - ")[1].split(' ')[0]
#     end_day = parts[0].split(" - ")[1].split(' ')[1]

#     start_date = f"{start_month} {start_day}, {year}"
#     end_date = f"{end_month} {end_day}, {year}"
    
#     print(start_date, end_date)

# else:
#     print('event_date not valid')

# # Clean up global variables used for testing
# # del test_date, test_date2, event_date, pattern, parts, year, month, start_day, end_day, start_date, end_date, start_month, end_month

In [120]:
def extract_start_end_dates(event_date, is_start):

    """
    Extracts the start or end date from a given event date string.
    The function handles two formats of event date strings:
    1. "Month Day - Month Day, Year" (e.g., "January 1 - January 3, 2023")
    2. "Month Day, Year" (e.g., "January 1-4, 2023")
    Args:
        event_date (str): The event date string to extract the start or end date from.
        is_start (bool): If True, returns the start date; if False, returns the end date.
    Returns:
        str: The extracted start or end date in the format "Month Day, Year".
    Raises:
        ValueError: If the event_date format is not recognized or parsing fails.
    """

    # pattern to determine which format the event date string is in
    pattern = r"\w+ \d+ - \w+ \d+, \d{4}"

    # if the event date string does not overlap months, then extract start and end dates
    if len(re.findall(pattern, event_date)) == 0:
        try:
            #split the passed date on comma to get the year and month/day sections
            parts = event_date.split(",")
            year = parts[-1].strip()
            month = parts[0].split(" ")[0]

            #extract the start and end days from the month/day section using regex by finding the digits
            start_day = re.findall("\d+", parts[0])[0]
            end_day = re.findall("\d+", parts[0])[1]

            start_date = f"{month} {start_day}, {year}"
            end_date = f"{month} {end_day}, {year}"
            
            #return the start or end date based on the is_start flag
            if is_start:
                return start_date
            else:
                return end_date
        except:
            print(f"this date failed {event_date}")
    
    # if the event date string overlaps months, then extract start and end dates        
    elif len(re.findall(pattern, event_date)) > 0:
        try:
            parts = event_date.split(",")
            year = parts[-1].strip()
            
            # extract the start and end months and days from the month/day section
            start_month = parts[0].split(" - ")[0].split(' ')[0]
            start_day = parts[0].split(" - ")[0].split(' ')[1]

            end_month = parts[0].split(" - ")[1].split(' ')[0]
            end_day = parts[0].split(" - ")[1].split(' ')[1]

            start_date = f"{start_month} {start_day}, {year}"
            end_date = f"{end_month} {end_day}, {year}"
            
            if is_start:
                return start_date
            else:
                return end_date
        except:
            print(f"this date failed {event_date}")
        



In [121]:
# applying functions to extract start and end dates
details_df['start_date'] = details_df['event_date'].apply(lambda x: extract_start_end_dates(x, True))
details_df['end_date'] = details_df['event_date'].apply(lambda x: extract_start_end_dates(x, False))

this date failed February 24, 2002
this date failed April 3, 1999
this date failed February 24, 2002
this date failed April 3, 1999


#### Convert date strings to date time

In [122]:
from datetime import datetime

def convert_event_date(event_date):
    """
    Converts an event date string to a datetime object.
    
    This function checks if the month in the event date string is abbreviated or not,
    and then converts the event date to a datetime object.
    
    Args:
        event_date (str): The event date string to convert.
        
    Returns:
        datetime: The converted datetime object.
    """
    # Define month formats
    full_months = [
        "January", "February", "March", "April", "May", "June",
        "July", "August", "September", "October", "November", "December"
    ]
    abbreviated_months = [
        "Jan", "Feb", "Mar", "Apr", "May", "Jun",
        "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
    ]
    
    # Convert to datetime object
    try:
        if event_date.split(" ")[0] in full_months:
            event_date_obj = datetime.strptime(event_date, "%B %d, %Y").date()
        elif event_date.split(" ")[0] in abbreviated_months:
            event_date_obj = datetime.strptime(event_date, "%b %d, %Y").date()
    
        return event_date_obj
    except:
        print(f"this date failed {event_date}")

In [123]:
# applying function to convert event dates to datetime objects
details_df['start_date'] = details_df['start_date'].apply(convert_event_date)
details_df['end_date'] = details_df['end_date'].apply(convert_event_date)

this date failed None
this date failed None
this date failed None
this date failed None


In [124]:
details_df['start_date'] = pd.to_datetime(details_df['start_date'])
details_df['end_date'] = pd.to_datetime(details_df['end_date'])


In [126]:
details_df

,event_id,tournament_title,event_date,golf_course,course_details,purse,city,Yards,Par,par_from_scores,par_final,golf_course_clean,start_date,end_date
0,1,WGC-Accenture Match Play Championship,"January 4 - 7, 2001",,,5000000,,NaN,NaN,NaN,NaN,,2001-01-04,2001-01-07
1,2,Mercedes Championships,"January 11 - 14, 2001","Kapalua Resort (Plantation Course) - Kapalua, HI",Par73Yards7411,3500000,"Kapalua, HI",7411.0,73.0,73.0,73.0,Kapalua Resort (Plantation Course),2001-01-11,2001-01-14
2,3,Touchstone Energy Tucson Open,"January 11 - 14, 2001",Omni Tucson National Golf Resort and Spa - Tuc...,Par72Yards7213,3000000,"Tucson, AZ",7213.0,72.0,72.0,72.0,Omni Tucson National Golf Resort and Spa,2001-01-11,2001-01-14
3,4,Sony Open in Hawaii,"January 18 - 21, 2001","Waialae Country Club - Honolulu, HI",Par70Yards7044,4000000,"Honolulu, HI",7044.0,70.0,70.0,70.0,Waialae Country Club,2001-01-18,2001-01-21
4,5,Phoenix Open,"January 25 - 28, 2001","TPC Scottsdale (Stadium Course) - Scottsdale, AZ",,4000000,"Scottsdale, AZ",NaN,NaN,71.0,71.0,TPC Scottsdale (Stadium Course),2001-01-25,2001-01-28
5,6,AT&T Pebble Beach National Pro-Am,"February 1 - 4, 2001",,Par72Yards6822,4000000,,6822.0,72.0,72.0,72.0,,2001-02-01,2001-02-04
6,7,Buick Invitational,"February 8 - 11, 2001",,,3500000,,NaN,NaN,NaN,NaN,,2001-02-08,2001-02-11
7,8,Bob Hope Chrysler Classic,"February 14 - 18, 2001",,Par71Yards6997,3500000,,6997.0,71.0,NaN,71.0,,2001-02-14,2001-02-18
8,9,Nissan Open,"February 22 - 25, 2001","Riviera Country Club - Pacific Palisades, CA",,3400000,"Pacific Palisades, CA",NaN,NaN,72.0,72.0,Riviera Country Club,2001-02-22,2001-02-25
9,10,Genuity Championship,"March 1 - 4, 2001","Trump National Doral Golf Course - Doral, FL",,4500000,"Doral, FL",NaN,NaN,72.0,72.0,Trump National Doral Golf Course,2001-03-01,2001-03-04


In [125]:
# import inspect

# # Get the current global namespace
# current_globals = globals()

# # Filter out functions from the global namespace
# functions_list = [name for name, obj in current_globals.items() if inspect.isfunction(obj)]

# # Display the list of function names
# functions_list
